# Project Manager Agent - Live Test

This notebook tests the `AgenticProjectManager` class using **REAL** dependencies and environment variables.

**Pre-requisites:**
- `.env` file must be present in the project root (`e:\Individual\Projects\ProMeet\.env`)
- Database specified in `DATABASE_URL` must be accessible.
- Google GenAI keys must be valid.

In [2]:
import sys
import os
from dotenv import load_dotenv

# 1. Setup Paths
current_dir = os.getcwd()
# Assuming this notebook is in ai_service/notebooks
# We need to reach the project root containing .env and ai_service/src

project_root = os.path.abspath(os.path.join(current_dir, '..', '..')) 
ai_service_root = os.path.abspath(os.path.join(current_dir, '..'))

if project_root not in sys.path:
    sys.path.append(project_root)
    
if ai_service_root not in sys.path:
    sys.path.append(ai_service_root)

print(f"Project Root: {project_root}")
print(f"AI Service Root: {ai_service_root}")

# 2. Load Environment Variables from Project Root
dotenv_path = os.path.join(project_root, '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)
    print(f"Loaded .env from {dotenv_path}")
else:
    print(f"WARNING: .env not found at {dotenv_path}")
    
# Optional: Check critical env vars
print(f"DATABASE_URL set: {bool(os.getenv('DATABASE_URL'))}")
print(f"GEMINI_API_KEY set: {bool(os.getenv('GEMINI_API_KEY'))}")

Project Root: e:\Individual\Projects\ProMeet
AI Service Root: e:\Individual\Projects\ProMeet\ai_service
Loaded .env from e:\Individual\Projects\ProMeet\.env
DATABASE_URL set: True
GEMINI_API_KEY set: True


In [4]:

import warnings
# Suppress specific deprecation warnings from libraries
warnings.filterwarnings("ignore", category=FutureWarning, module="langchain_google_genai")# 3. Import and Initialize Agent
try:
    from src.agents.project_manager.agent import AgenticProjectManager, AgentState
    print("Agent module imported successfully.")
    
    # Initialize
    # NOTE: This will attempt to connect to Postgres via psycopg_pool.
    agent = AgenticProjectManager()
    print("Agent initialized successfully.")
except Exception as e:
    print(f"Error initializing agent: {e}")
    import traceback
    traceback.print_exc()

Agent module imported successfully.
Agent initialized successfully.


In [5]:
# 4. Test Interaction Loop
def chat_with_agent(query_text):
    print(f"\nUser: {query_text}")
    
    state = {
        "messages": [],
        "query": query_text,
        # 'router_decision' will be filled by the graph
        "router_decision": ""
    }
    
    # Invoke the graph
    # Using .invoke on the compiled graph
    try:
        # config dictionary is usually needed for Checkpointer if checkpointer is enabled
        # The agent uses PostgresSaver, so it might need a thread_id
        config = {"configurable": {"thread_id": "test_thread_1"}}
        
        result = agent.graph.invoke(state, config=config)
        
        # Extract last message
        if 'messages' in result and result['messages']:
            last_msg = result['messages'][-1]
            print(f"Agent: {last_msg.content}")
            
            # If tool calls existed in the history, print them
            for msg in result['messages']:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    print(f"(Tool Call Attempt: {msg.tool_calls})")
        else:
            print("Agent returned no messages.")
            
    except Exception as e:
        print(f"Error during invocation: {e}")
        import traceback
        traceback.print_exc()

# Test 1: Direct small talk
chat_with_agent("Tôi tên là Phước, bạn có khỏe không?")
chat_with_agent("Tôi tên gì?")
# Test 2: Project query (Assuming DB has data, otherwise this tests the logic)
# This will trigger router -> TOOL_CALL -> tool_generator -> ...
# chat_with_agent("Danh sách dự án của tôi")


User: Tôi tên là Phước, bạn có khỏe không?
Agent: Chào Phước, tôi khỏe, cảm ơn bạn. Còn bạn thì sao?

User: Tôi tên gì?
Agent: Bạn tên là Phước.
